In [1]:
import numpy as np
import torch
import os
import sys
sys.path.append(os.path.join(os.path.split(os.getcwd())[0],'data_loader'))
# from data import DataBuildClassifier
from bg.data_proj_pursuit_v2 import DataProjPursuit_v2 as DataBuildClassifier
from src.model_torch import EEGNet_experimental
from scipy.signal import welch
from scipy.fftpack import fft,rfft, fftshift, fftfreq, rfftfreq
import mne
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [2]:
params = {'resample_to': 369,
                 'D': 3,
                 'F1': 12,
                 'dropoutRate1': 0.52,
                 'dropoutRate2': 0.36,
                 'lr': 0.00066,
                 }

In [3]:
# model = torch.load('/home/likan_blk/BCI/eegnet_torch/res/cv_simple_ebci_torch/EEGNET_v4/25/naive_model')
model = torch.load('./res/cv_simple_ebci_torch/EEGNET_v4/25/naive_model')

In [4]:
# data = DataBuildClassifier('/home/likan_blk/BCI/NewData')
data = DataBuildClassifier('../new_data')
# all_subjects = [25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38]
all_subjects = 30
# subjects = data.get_event_data(all_subjects, shuffle=False, window=[(0.2, 0.5)], baseline_window=(0.2, 0.3),
#                              resample_to=params['resample_to'], event='eegNT')
subjects = data.get_event_data(all_subjects, shuffle=False, window=[0.1, 0.2], baseline_window=(0.2, 0.3),
                             event='eegT')

[[[  6132.56899121   6106.91963261   6114.23528245 ...   3167.0335267
     3115.40139072   3128.03050884]
  [ -4435.91337818  -4492.50355724  -4481.32415117 ...  -5420.40952411
    -5433.64923536  -5408.46851658]
  [  1097.25431097   1066.44850444   1085.12809166 ...   -784.47003199
     -781.84863052   -785.18014418]
  ...
  [-11381.6070643  -11402.47969836 -11382.40944467 ...  -9247.46589201
    -9258.34901681  -9224.0126583 ]
  [  1423.38210871   1394.5912434    1406.20361474 ...    546.62752406
      534.90691203    568.07153625]
  [  4038.18554745   4000.58624933   4004.65807642 ...   2001.0432259
     1989.29209594   2001.96414597]]

 [[  6132.89080513   6106.18423481   6112.6064757  ...   3165.80144977
     3113.91501289   3127.88274593]
  [ -4435.57113423  -4493.30815206  -4481.71026494 ...  -5421.8056273
    -5435.00009277  -5408.72808242]
  [  1098.07054177   1066.55256633   1083.82088027 ...   -786.09204373
     -783.6215538    -785.57604026]
  ...
  [-11381.522337   -11403.

E:\MIPT\kurchatov\diploma\ZHAO\bg\data_proj_pursuit_v2.py:18: RuntimeWarning: Mean of empty slice.
  baseline = np.expand_dims(X[:, bl_start:bl_end, :].mean(axis=1), axis=1)
C:\Users\vaa09\anaconda3\envs\nlp_course\lib\site-packages\numpy\core\_methods.py:78: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [5]:
x,y = subjects[25][0], subjects[25][1]
x = x.transpose(0,2,1)[:,np.newaxis,:,:]

ValueError: axes don't match array

In [ ]:
kernLength1 = params.get('kernLength1',64)
kernLength2 = params.get('kernLength2',16)
F2 = params.get('F2',params['F1']*params['D'])
model = EEGNet_experimental(nb_classes=2, Chans=subjects[25][0].shape[2], Samples = subjects[25][0].shape[1],
                               dropoutRates=(params['dropoutRate1'], params['dropoutRate1']),
                               kernLength1=kernLength1, kernLength2=kernLength2, poolKern1=4,
                               poolKern2=8,
                               F1=params['F1'],
                               D=params['D'], F2=F2)
model.load_state_dict(torch.load('/home/likan_blk/BCI/eegnet_torch/res/cv_simple_ebci_torch/EEGNET_v4/25/naive_model'))

In [ ]:
spatial_filters = model.depthwise1.weight.cpu().detach().numpy()

In [ ]:
def read_info(path_to_file):
    with open(path_to_file,'r') as f:
        content = f.readlines()
    content = content[1:]
    print(len(content))
    res = np.zeros((len(content),2))
    for i in range(len(content)):
        splt_line = content[i].split()
        res[i,0] = np.pi*float(splt_line[1])/180 + np.pi/2
        res[i,1] = float(splt_line[2])
    return res
info = read_info('/home/likan_blk/BCI/NewData/order&locations.info')

In [ ]:
plt.polar(info[:,0],info[:,1],'ro')

In [ ]:
def pol2cart(phi,r ):
    x = r * np.cos(phi)
    y = r * np.sin(phi)
    return np.vstack((x, y)).T
cort_info = pol2cart(info[:,0], info[:,1])

plt.scatter(cort_info[:,0],cort_info[:,1])

In [ ]:
spatial_filters.shape

In [ ]:
mne.viz.plot_topomap(spatial_filters[0,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[1,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[2,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[3,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[4,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[5,0,:,0],cort_info)
# mne.viz.plot_topomap(spatial_filters[6,0,:,0],cort_info)

In [ ]:
first_output = model.conv1(torch.Tensor(torch.Tensor(x)))
first_output.shape

In [ ]:
weights = model.conv1.weight.detach().cpu().numpy()[:,0,0,:]

In [ ]:
weights.shape

In [ ]:
yf = fft(weights[4,:])
xf = fftfreq(weights.shape[1], 1/params['resample_to'])
xf = fftshift(xf)
yplot = fftshift(yf)
plt.plot(xf, 1.0/weights.shape[1] * np.abs(yplot))

In [ ]:
fft(weights[4,:])

In [ ]:
rfft(weights[4,:])